### 4th European Conference On Social Networks
## Workshop: Introduction to Python's Graph-Tool
# 1. Graph Handling and Vertex-Level Analysis
**Authors**: <a href='https://www.gesis.org/person/haiko.lietz'>Haiko Lietz</a>, <a href='https://www.gesis.org/person/marcos.oliveira'>Marcos Oliveira</a>, GESIS - Leibniz Institute for the Social Sciences

**Version Date**: 9 September 2019 

**Description**: This notebook introduces graph loading, drawing, transformation, filtering and viewing, and centrality analysis.

**License**: <a href='https://www.gnu.org/licenses/gpl-3.0.en.html'>GNU General Public License 3.0</a>

***

In [ ]:
from graph_tool.all import *

In [ ]:
openmp_set_num_threads(2)

# 1.1. Graph Loading
The example used here is the network of influential families in Renaissance Florence (<a href='https://doi.org/10.1086/230190'>Padgett & Ansell, 1993</a>). It is directed, weighted, multi-layered, and has node attributes. Hence, it is a good training example for handling all this information.
<img src='images/medici.jpg'>

In [ ]:
import pandas as pd

The Medici dataset has rich node metadata:

In [ ]:
node_list = pd.read_csv('data/medici/node_list.txt', header='infer', delimiter='\t', encoding='utf-8')
node_list

### 1.1.1. Quickly Loading a Graph From File
A quick way to load network data is to have an edge list with edge attributes ready to go:

In [ ]:
edge_list_string = pd.read_csv('data/medici/edge_list_string.txt', header='infer', delimiter='\t', encoding='utf-8')
edge_list_string.head()

Edges have two properties, a weight and an identifier to which of four domains it belongs: kinship (value ``0``), economic (``1``), political (``2``), or friendship (``3``). graph-tool handles edges that belong to multiple domains as parallel edges, without the necessity to specify a multiplex graph. Edges are directed. Since friendship is originally undirected, both directions, from ``i`` to ``j`` and from ``j`` to ``i`` are included in the edge list.

Data of this form can be used directly to load a graph via the <a href='https://graph-tool.skewed.de/static/doc/graph_tool.html?highlight=csv#graph_tool.load_graph_from_csv'>load_graph_from_csv()</a> function. Make sure to specify if the graph is ``directed``, what ``eprop_types`` and ``eprop_names`` the edge properties have, if the first row in the file should be skipped (``skid_first=True``) because it contains a header, and what ``delimiter`` is used. Remember that, in graph-tool, each vertex in a graph has an unique index, which is *always* between $0$ and $N−1$, where $N$ is the number of vertices. In our case, the vertex values are arbitrary strings that must be mapped to integers (set ``string_vals=True`` and ``hashed=True``).

In [ ]:
g = load_graph_from_csv('data/medici/edge_list_string.txt', directed=True, eprop_types=['int', 'int'], eprop_names=['weight', 'domain_id'], string_vals=True, hashed=True, skip_first=True, csv_options={'delimiter': '\t'})

In [ ]:
g.num_vertices()

In [ ]:
g.num_edges()

The <a href='https://graph-tool.skewed.de/static/doc/graph_tool.html?highlight=csv#graph_tool.load_graph_from_csv'>load_graph_from_csv()</a> function internalizes edge properties and vertex names as property maps:

In [ ]:
g.list_properties()

To return the actual array, use the <a href='https://graph-tool.skewed.de/static/doc/graph_tool.html?highlight=property%20map#graph_tool.PropertyMap.get_array'>get_array()</a> and <a href='https://graph-tool.skewed.de/static/doc/graph_tool.html?highlight=property%20map#graph_tool.PropertyMap.get_2d_array'>get_2d_array()</a> functions:

In [ ]:
g.vp.name.get_2d_array(pos=[0]) # vp stands for vertex property, "name" is the name of the vertex property

In [ ]:
g.ep.domain_id.get_array() # shortcut: g.ep.domain_id.a

In [ ]:
g.ep.weight.a

These property maps can be used when drawing the graph using <a href='https://graph-tool.skewed.de/static/doc/draw.html?highlight=graph_draw#graph_tool.draw.graph_draw'>graph_draw()</a>:

In [ ]:
graph_draw(g, edge_pen_width=g.ep.weight, edge_color=g.ep.domain_id, output_size=(400, 300)) # use vertex_text=g.vp.name to label the nodes

Note that the indices and names of vertices in the graph don't match those in the ``node_list`` - a consequence of hashing vertex strings. This can be avoided by preparing an edge list that uses vertex integers that are in accordance with the node list:

In [ ]:
edge_list_int = pd.read_csv('data/medici/edge_list_int.txt', header='infer', delimiter='\t', encoding='utf-8')
edge_list_int.head()

### 1.1.2. The Detailed Way to Load a Graph
The quick way to load a graph does not load vertex properties as available in the ``node_list``. The detailed way to load a graph is to load node and edge lists from data frames, not from file. First, an empty <a href='https://graph-tool.skewed.de/static/doc/graph_tool.html?highlight=graph#graph_tool.Graph'>Graph</a> object is created:

In [ ]:
h = Graph(directed=True)

In [ ]:
h.is_directed()

The first step in populating the Graph object with vertices is to specify the properties the nodes should have:

In [ ]:
vp_pos = h.new_vertex_property('vector<double>') # vertex position
vp_party = h.new_vertex_property('string') # party as vertex shape
vp_color = h.new_vertex_property('vector<double>') # partisanship as vertex fill color
vp_wealth = h.new_vertex_property('int') # family wealth as vertex size
vp_family = h.new_vertex_property('string') # family name as vertex label

The second step is to loop through all vertices defined in the ``node_list`` and add the respective properties to the maps:

In [ ]:
for i in range(0, len(node_list)):
    v = h.add_vertex()
    vp_pos[v] = [node_list['pos_x'][i], node_list['pos_y'][i]]
    vp_party[v] = 'square' if node_list['party'][i] == 'MEDICI' else 'circle'
    vp_color[v] = [node_list['color_r'][i], node_list['color_g'][i], node_list['color_b'][i], 1]
    vp_wealth[v] = node_list['wealth'][i]
    vp_family[v] = node_list['family'][i]

The ``vp_color`` property map contains values for the fraction of red, green, blue (each in the $[0,1]$ interval), and a transparency score (in the same interval):

In [ ]:
vp_color.get_2d_array([0, 1, 2, 3])

The maps are still external to the Graph object. The third step is to internalize them:

In [ ]:
h.vertex_properties['pos'] = vp_pos
h.vertex_properties['party'] = vp_party
h.vertex_properties['color'] = vp_color
h.vertex_properties['wealth'] = vp_wealth
h.vertex_properties['family'] = vp_family

Populating the graph with edges is quite similar. First specify the desired edge properties:

In [ ]:
ep_weight = h.new_edge_property('int') # weight as edge pen width
ep_domain = h.new_edge_property('int') # domain
ep_color = h.new_edge_property('vector<double>') # domain as edge color

Second, add the edges. This time, we can use the handy <a href='https://graph-tool.skewed.de/static/doc/graph_tool.html?highlight=add_edge_list#graph_tool.Graph.add_edge_list'>add_edge_list()</a> function. The parameter ``eprops`` must be a list of edge property names ordered as in the dataframe:

In [ ]:
h.add_edge_list(edge_list_int.values, eprops=[ep_weight, ep_domain])

In [ ]:
n = h.num_vertices()
n

The ``ep_color`` edge property map should contain a domain-specific color defined in this dictionary:

In [ ]:
dict_color = {0: [0.6, 0.31, 0.64, 1], 1: [1, 0.5, 0, 1], 2: [0.3, 0.69, 0.29, 1], 3: [0.65,0.34,0.16,1]}

Loop through the edges and assign colors to the ``ep_color`` property map:

In [ ]:
for e in h.edges():
    ep_color[e] = dict_color[ep_domain[e]]    

The third step is again to internalize the needed edge property maps:

In [ ]:
h.edge_properties['weight'] = ep_weight
h.edge_properties['domain'] = ep_domain
h.edge_properties['color'] = ep_color

## 1.2. Graph Drawing
Much metadata can be visualized beyond the graph itself. List the available metadata that we have internalized as property maps:

In [ ]:
h.list_properties()

Draw the graph using the <a href='https://graph-tool.skewed.de/static/doc/draw.html?highlight=gradient%20color#graph_tool.draw.graph_draw'>graph_draw()</a> function. Internalized property maps, e.g., for parameters like ``pos`` and ``vertex_shape``, are called by ``<graph_name>.vp.<property_name>`` and ``<graph_name>.ep.<property_name>`` for vertex properties and edge properties, respectively. To rescale vertex sizes, use the <a href='https://graph-tool.skewed.de/static/doc/draw.html?highlight=prop_to_size#graph_tool.draw.prop_to_size'>prop_to_size()</a> function:

In [ ]:
graph_draw(h, 
           pos=h.vp.pos, 
           vertex_shape=h.vp.party, 
           vertex_fill_color=h.vp.color, 
           vertex_size=prop_to_size(h.vp.wealth, mi=5, ma=20), 
           #vertex_text=h.vp.family, 
           edge_pen_width=h.ep.weight, 
           edge_color=h.ep.color, 
           edge_marker_size=10, 
           output_size=(600, 450)) # to save the plot, set output='h.pdf' or output='h.png'

The above layout uses given node positions that are close to those used in the original paper. If no ``pos`` parameter is set, <a href='https://graph-tool.skewed.de/static/doc/draw.html?highlight=gradient%20color#graph_tool.draw.graph_draw'>graph_draw()</a> will use the <a href='https://graph-tool.skewed.de/static/doc/draw.html?highlight=gradient%20color#graph_tool.draw.sfdp_layout'>sfdp_layout()</a> layout algorithm by default:

In [ ]:
graph_draw(h, vertex_shape=h.vp.party, vertex_fill_color=h.vp.color, vertex_size=prop_to_size(h.vp.wealth, mi=5, ma=20), edge_pen_width=h.ep.weight, edge_color=h.ep.color, edge_marker_size=10, output_size=(600, 450))

Other layout algorithms are listed in the documentation of the <a href='https://graph-tool.skewed.de/static/doc/draw.html'>draw</a> module. To use, e.g., a <a href='radial_tree_layout()'>radial_tree_layout()</a> with the Guasconi family as ``root=17`` node, store the  newly computed positions in a property map and call that in <a href='https://graph-tool.skewed.de/static/doc/draw.html?highlight=gradient%20color#graph_tool.draw.graph_draw'>graph_draw()</a>. Note that now an external property map is called:

In [ ]:
h_rtl = radial_tree_layout(h, root=17, weighted=True)

In [ ]:
graph_draw(h, pos=h_rtl, vertex_shape=h.vp.party, vertex_fill_color=h.vp.color, vertex_size=prop_to_size(h.vp.wealth, mi=5, ma=20), edge_pen_width=h.ep.weight, edge_color=h.ep.color, edge_marker_size=10, output_size=(600, 450))

## 1.3. Graph Transformation
Graphs can also contain information that must be removed or transformed, either for research purposes or for a certain algorithm to be applicable. For such purposes, the <a href='https://graph-tool.skewed.de/static/doc/stats.html'>stats</a> module has some useful functions.
### 1.3.1. Removing Parallel Edges
To remove the multi-layer information on domains, one can use <a href=''>remove_parallel_edges()</a>, but the function simply keeps "one edge from each parallel edge set". Since it cannot be controlled which edge is kept, e.g., the one with a stronger weight, edge weights are not meaningful anymore:

In [ ]:
i = h.copy() # work with a deep copy of h

In [ ]:
remove_parallel_edges(i)

In [ ]:
graph_draw(i, pos=i.vp.pos, vertex_shape=i.vp.party, vertex_fill_color=i.vp.color, vertex_size=prop_to_size(i.vp.wealth, mi=5, ma=20), edge_marker_size=10, output_size=(600, 450))

To obtain the sum of all edge weights (and keep meaningful edge weights), one can remove all edges from the copy of a graph and load a transformed edge list:

In [ ]:
j = h.copy()
j.clear_edges() # remove all edges
j.shrink_to_fit() # free memory if many edges have been removed
# delete internal edge property maps
del j.edge_properties['color']
del j.edge_properties['weight']

Transform the edge list...

In [ ]:
edge_list_int_remove_parallel = edge_list_int[['family_id_i', 'family_id_j', 'weight']].groupby(['family_id_i', 'family_id_j']).sum().reset_index()

... and load it:

In [ ]:
ep_weight_remove_parallel = j.new_edge_property('int')
j.add_edge_list(edge_list_int_remove_parallel.values, eprops=[ep_weight_remove_parallel])
j.edge_properties['weight'] = ep_weight_remove_parallel

In [ ]:
graph_draw(j, pos=j.vp.pos, vertex_shape=j.vp.party, vertex_fill_color=j.vp.color, vertex_size=prop_to_size(j.vp.wealth, mi=5, ma=20), edge_pen_width=j.ep.weight, edge_marker_size=10, output_size=(600, 450))

### 1.3.2. Removing Edge Direction
Given a weighted directed graph such as ``j``, there are a couple of ways how directed edges can be transformed into undirected ones, e.g., taking the maximum, mimimum, average, sum, etc. of bidirectional edges. Such transformations are currently not available in graph-tool. The only option to symmetrize a directed graph is to remove the direction, which creates parallel edges:

In [ ]:
j.is_directed()

In [ ]:
j.set_directed(False)

In [ ]:
graph_draw(j, pos=j.vp.pos, vertex_shape=j.vp.party, vertex_fill_color=j.vp.color, vertex_size=prop_to_size(j.vp.wealth, mi=5, ma=20), edge_pen_width=j.ep.weight, edge_marker_size=10, output_size=(600, 450))

In [ ]:
k = j.copy()
remove_parallel_edges(k)

Again, due to uncontrollable edge removal, edge weights have lost their meaning:

In [ ]:
graph_draw(k, pos=k.vp.pos, vertex_shape=k.vp.party, vertex_fill_color=k.vp.color, vertex_size=prop_to_size(k.vp.wealth, mi=5, ma=20), edge_marker_size=10, output_size=(600, 450))

## 1.4. Graph Filtering and Viewing
In fact, in the symmatrized graph ``j``, edge directions are only filtered out through the ``set_directed(False)`` function. Since the arrows are only temporarily masked, direction can be recovered:

In [ ]:
j.set_directed(True)

In [ ]:
graph_draw(j, pos=j.vp.pos, vertex_shape=j.vp.party, vertex_fill_color=j.vp.color, vertex_size=prop_to_size(j.vp.wealth, mi=5, ma=20), edge_pen_width=j.ep.weight, edge_marker_size=10, output_size=(600, 450))

Instead of filtering directed edges, it is possible to create an undirected "view" of ``j`` - a <a href='https://graph-tool.skewed.de/static/doc/graph_tool.html#graph_tool.GraphView'>GraphView</a> - that shares the underlying data with ``j``:

In [ ]:
j_view_undirected = GraphView(j, directed=False)

In [ ]:
graph_draw(j_view_undirected, pos=j_view_undirected.vp.pos, vertex_shape=j_view_undirected.vp.party, vertex_fill_color=j_view_undirected.vp.color, vertex_size=prop_to_size(j_view_undirected.vp.wealth, mi=5, ma=20), edge_pen_width=j_view_undirected.ep.weight, edge_marker_size=10, output_size=(600, 450))

The real power of graph views, however, is their use in combination with filters. The idea is that vertices or edges that shoud be masked (for drawing and analysis) are marked with a boolean property map where ``True`` indicates that the respective vertex or edge should be kept. This way we can, e.g., draw the four domain layers of the Medici graph:

In [ ]:
for d in range(0, 4):
    domain = h.ep.domain.a == d
    h_view_domain = GraphView(h, efilt=domain)
    graph_draw(h_view_domain, pos=h_view_domain.vp.pos, vertex_shape=h_view_domain.vp.party, vertex_fill_color=h_view_domain.vp.color, vertex_size=prop_to_size(h_view_domain.vp.wealth, mi=5, ma=20), edge_pen_width=h_view_domain.ep.weight, edge_color=h_view_domain.ep.color, edge_marker_size=10, output_size=(600, 450))

## 1.5. Centrality Analysis
Often we want to assign a value to each node, e.g., for ranking purposes. Centrality analysis does this only depending on the structure of the graph. The <a href='https://graph-tool.skewed.de/static/doc/centrality.html'>centrality</a> module lists nine centrality indices available in graph-tool. There are three families of centrality indices: degree-like, closeness-like, and betweenness-like (<a href='https://doi.org/10.1016/j.socnet.2005.11.005'>Borgatti & Everett, 2006</a>).
### 1.5.1. Degree Centrality
Degree centrality, not listed there, is actually the number of a node $x$'s neighbors, or the number of walks starting or ending at a node. For a weighted directed graph, there are six possible degree centralities. The get_out_degrees() and get_in_degrees() functions take as inut a list of vertices ``vs`` for which to get the degree and, optionally to get weighted degrees, an edge property map holding edge weights:

In [ ]:
h.get_out_degrees(vs=h.get_vertices(), eweight=None)

In [ ]:
h.get_in_degrees(vs=h.get_vertices(), eweight=None)

In [ ]:
h.get_out_degrees(vs=h.get_vertices(), eweight=h.ep.weight)

In [ ]:
h.get_in_degrees(vs=h.get_vertices(), eweight=h.ep.weight) # weighted in-degree

In graph-tool, the total degree of a directed graph (here: ``h``) corresponds to the out-degree of the corresponding undirected graph (to be created: ``h_view_undirected``):

In [ ]:
h_view_undirected = GraphView(h, directed=False)

In [ ]:
h_view_undirected.get_out_degrees(vs=h_view_undirected.get_vertices(), eweight=None)

In [ ]:
h_view_undirected.get_out_degrees(vs=h_view_undirected.get_vertices(), eweight=h_view_undirected.ep.weight)

To normalize degree centrality, it must be divided by the number of nodes $n$ in the graph.

To add the normalized total degree to graph ``h`` as a vertex property ``total_degree``, one can loop through all vertices of the view ``h_view_undirected`` and store their normalized degree.

In [ ]:
vp_total_degree = h.new_vertex_property('double')
for i in range(0, n):
    v = h_view_undirected.vertex(i)
    vp_total_degree[v] = v.out_degree()/n
h.vertex_properties['total_degree'] = vp_total_degree

### 1.5.2. Closeness Centrality
<a href='https://graph-tool.skewed.de/static/doc/centrality.html#graph_tool.centrality.closeness'>Closeness centrality</a> is the inverse of the sum of a node $x$'s distance $d$ to other nodes, where $d$ is the number of walks to all nodes but $x$ considering their length. Normalization is turned on by default. Pass a property map to the ``vprop`` parameter to directly store the values internally:

In [ ]:
h.vertex_properties['closeness'] = h.new_vertex_property('double')
closeness(h_view_undirected, vprop=h.vp.closeness, norm=True)

### 1.5.3. Betweenness Centrality
<a href='https://graph-tool.skewed.de/static/doc/centrality.html#graph_tool.centrality.betweenness'>Betweenness centrality</a> is the sum of the fraction of all-pairs shortest paths that pass through a node $x$, or the number of shortest walks through $x$. Normalization is turned on by default. Pass a property map to the ``vprop`` parameter to directly store the values internally. Note that the function returns property maps for vertices and edges:

In [ ]:
h.vertex_properties['betweenness'] = h.new_vertex_property('double')
h.edge_properties['betweenness'] = h.new_edge_property('double')
betweenness(h_view_undirected, vprop=h.vp.betweenness, eprop=h.ep.betweenness, norm=True)

To compare the values on the nodes in a plot, we reduce all other information:

In [ ]:
remove_parallel_edges(h_view_undirected)

In [ ]:
graph_draw(h_view_undirected, pos=h.vp.pos, vertex_fill_color='black', vertex_size=prop_to_size(h.vp.total_degree, mi=5, ma=20), edge_color='black', output_size=(400, 300))

In [ ]:
graph_draw(h_view_undirected, pos=h.vp.pos, vertex_fill_color='black', vertex_size=prop_to_size(h.vp.closeness, mi=5, ma=20), edge_color='black', output_size=(400, 300))

In [ ]:
graph_draw(h_view_undirected, pos=h.vp.pos, vertex_fill_color='black', vertex_size=prop_to_size(h.vp.betweenness, mi=5, ma=20), edge_pen_width=prop_to_size(h.ep.betweenness, mi=1, ma=4), edge_color='black', output_size=(400, 300))

## 1.6. Exercise
Graph-tool also provides a <a href='https://graph-tool.skewed.de/static/doc/collection.html'>collection</a> of graphs. A graph from the collection is loaded like this:

In [ ]:
l = collection.data['netscience']

In [ ]:
l.list_properties()

Analyze this graph using the tools of this notebook...